In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, recall_score, precision_score, det_curve
from evaluate_metrics import compute_eer, compute_cllr, compute_min_dcf, calculate_minDCF_EER_CLLR_actDCF
import librosa
from IPython.display import Audio, display
import librosa.display
import matplotlib.pyplot as plt
import random
def recursive_list_files(directory, extension=None):
    """
    Recursively list files in a directory
    :param directory: directory to list files from
    :param extension: file extension to filter by
    :return: list of files in directory
    """
    files = []
    for root, dirs, filenames in os.walk(directory, followlinks=True):
        for filename in filenames:
            if extension is None or filename.endswith(extension):
                files.append(os.path.join(root, filename))
    return files

def play_and_show(file_path):
    """
    Play and show an audio file
    :param file_path: path to audio file
    """
    y, sr = librosa.load(file_path, sr=None)
    fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True)
    display(Audio(file_path, rate=16000))
    # Compute the spectrogram
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=240, win_length=480, window='hamming'))
    D = librosa.amplitude_to_db(S, ref=np.max)
    img = librosa.display.specshow(D, y_axis='linear', x_axis='time', n_fft=2048, hop_length=240, win_length=480, 
                               sr=sr, ax=ax)
    plt.show()

    

/home/phucdt/anaconda3/envs/asvspoof5/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
datadir = '/dataa/phucdt/vocodetect/traindata/CyberAICup2024/rTTSD_Train'
protocol = pd.read_csv('/dataa/phucdt/vocodetect/traindata/CyberAICup2024/my_rTTSD_train_list_protocol.txt', sep=' ', header=None)
protocol.columns = ['utt', 'subset', 'label']


In [3]:
# calculate the duration and add it to the protocol
duration = []
for idx, row in tqdm(protocol.iterrows(), total=len(protocol)):
    file_path = os.path.join(datadir, row['utt'])
    y, sr = librosa.load(file_path, sr=None)
    duration.append(len(y) / sr)
protocol['duration'] = duration


100%|██████████| 33358/33358 [12:58<00:00, 42.87it/s]


In [4]:
protocol.to_csv('/dataa/phucdt/vocodetect/traindata/CyberAICup2024/my_rTTSD_train_list_protocol_duration.csv', index=False)